<img src='https://www.icos-cp.eu/sites/default/files/2017-11/ICOS_CP_logo.png' width=400 align=right>
<img src='https://qcdis.github.io/NaaVRE-website/img/hero-light.svg' width=200 align=right>


# ICOS Carbon Portal Python Library in NaaVRE

## Documentation

- [ICOS Carbon Portal Python Library](https://icos-carbon-portal.github.io/pylib/)
- [NaaVRE](https://qcdis.github.io/NaaVRE-website/) (Notebook as a Virtual Research Environment)

## Install libraries

In [ ]:
!pip install icoscp matplotlib python-slugify

## NaaVRE workflow parameters

To get an ICOS API token for `param_cpauth_token`: go to https://cpauth.icos-cp.eu, login (creating an account if needed), accept the ICOS data license, and copy the API token at the bottom of the page (`cpauthToken=xyzABC...`).

In [ ]:
# (Do not containerize, NaaVRE workflow parameters cell)

param_cpauth_token = ''

param_ecosystem_type = 'Wetland'
param_data_type = 'ETC L2 Fluxes'
param_variable = 'CO2'

 ## List stations

In [ ]:
# List ICOS stations

from icoscp.station import station

stations = station.getIdList()
stations = stations[stations.siteType == param_ecosystem_type]
stations_id_list = list(stations.id)

stations_id_list

## List data products

In [ ]:
# List data products

import warnings
import pandas as pd

dobj_list = []
for station_id in stations_id_list:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        s = station.get(station_id)
        datasets = s.data()
    if not isinstance(datasets, pd.DataFrame):
        print(f'No datasets for station {station_id}')
        continue
    datasets = datasets[datasets.specLabel == param_data_type]
    dobj_list += list(datasets.dobj)

dobj_list

## Plot data

In [ ]:
# Plot time series

from icoscp.cpauth.authentication import Authentication
from icoscp.cpb.dobj import Dobj
import matplotlib.pyplot as plt
import slugify

Authentication(token=param_cpauth_token)

plot_files = []
for dobj_pid in dobj_list:
    dobj = Dobj(dobj_pid)
    unit = dobj.variables[dobj.variables.name == param_variable].unit.values[0]
    name = dobj.station['org']['name']
    uri = dobj.station['org']['self']['uri']
    title = f"{name} \n {uri}"
    plot = dobj.data.plot(x='TIMESTAMP', y=param_variable, grid=True, title=title)
    plot.set(ylabel=unit)
    filename = f'/tmp/data/{slugify.slugify(dobj_pid)}.pdf'
    plt.savefig(filename)
    plot_files.append(filename)
    plt.show()

plot_files

## Upload plots (optional)

To retrieve the generated files after the workflow has completed, they need to be uploaded to a persistent storage location. For testing purposes, we provide a S3-compatible MinIO server.

Follow this [tutorial](https://docs.google.com/presentation/d/112Vs-vsOonVq1TlC4WprzWR6s9XntOBGgoFGbbT_FDg) (part "Minio Setup", slide 2 to 9) to generate `param_s3_user_prefix`, `param_s3_access_key` and `param_s3_secret_key`.

In [ ]:
# (Do not containerize, NaaVRE workflow parameters cell)

param_s3_server = "scruffy.lab.uvalight.net:9000"
param_s3_bucket = "naa-vre-user-data"

param_s3_user_prefix = ""
param_s3_access_key = ""
param_s3_secret_key = ""

In [ ]:
# Upload files

import os
from minio import Minio

minio_client = Minio(param_s3_server, access_key=param_s3_access_key, secret_key=param_s3_secret_key, secure=True)

for plot_file  in plot_files:
    print("Uploading", plot_file)
    object_name = f'{param_s3_user_prefix}/vl-openlab/icos-naavre-demo/{os.path.basename(plot_file)}'
    minio_client.fput_object(param_s3_bucket, object_name=object_name, file_path=plot_file)